In [381]:
import random
import io
import string
from sklearn.metrics.pairwise import cosine_similarity

In [376]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [321]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

In [303]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [319]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 

import MeCab 
mecab = MeCab.Tagger('-Owakati')

p = Path("/home/ifte/Downloads/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv(p, header=0, usecols=["page", "text"])

In [322]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [334]:
classes = df.page.unique()
c_size = 100

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    data = ''.join(list(arr_index))
    
    all_data = clean_text(data)
    
    arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    num = arr_words.shape[0]//c_size
    full = c_size * num
    rest = arr_words.shape[0] - full
    
    pad = np.zeros([c_size-rest], dtype=int)
    sc = np.concatenate((arr_words,pad))
    features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(features)
    df_v["cls"] = i
    
    result = pd.concat([result, df_v]) 

    
#print(result)
#result.to_csv('processed.csv')

In [396]:
Data_cols = result.iloc[:,:-1]
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
#tog = result.iloc[:,:].apply(lambda x: ' '.join(x), axis=1)
sent = list(tog.values)

In [397]:
def response(user_response):
    robo_response=''
    sent.append(user_response)
    TfidfVec = TfidfVectorizer(stop_words=stop_words_ja)
    tfidf = TfidfVec.fit_transform(sent)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+tog.values[idx]
        return robo_response


In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")    


ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
showmessagebox what is this
ROBO: あれ ば その 変更 を 要求 する rem または ; 説明 Remark 複数 行 の スクリプト 文 で 説明 文 を 挿入 する 際 に 使う サンプル rem this is a comment ; this is a comment reset 説明 端末 を リセット し ます サンプル reset s delay 秒 後 に Soft Reset 構文 reset < switch > delay < 秒 > reset Soft Reset 端末 電源 を 落とさ ず に 再 起動 リモートコントロール 状態 なら リモートコントロール を 停止 する reset s Soft Reset 端末 電源 を 落とさ ず に 再 起動 リモートコントロール 状態 なら リモートコントロール を 停止 する reset
showmessagebox 
ROBO: タイトル の アイコン に 種類 が あり ます 情報 警告 質問 エラー サンプル showmessagebox " 在庫 検索 アプリ の バージョンアップ を し まし た " NO TIMER 「 OK 」 ボタン が 表示 さ れ 端末 ユーザ が タップ する まで 表示 継続 サンプル showmessagebox " 今 から バージョンアップ を 開始 し ます " 「 OK 」 ボタン が 表示 さ れ 端末 ユーザ が タップ する まで 表示 継続 端末 ユーザ が タップ し ない と 秒 後 に 表示 終了 サンプル showmessagebox " 今 から バージョンアップ を 開始 し ます " 「 OK 」 ボタン が 表示 さ


In [301]:
df = result.sample(frac=1).reset_index(drop=True)

In [293]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


ValueError: Found input variables with inconsistent numbers of samples: [23910, 797]